# Notebook to explore lat/long position of wells & find nearest neighbors
### and potentially functions to make a dataset of nearest neighbors within certain limits for each well
### and potentially functions to make a dataset of tops and other characteristics for neighboring wells.

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
welly.__version__

'0.3.0'

In [2]:
%%timeit
import os
env = %env

97.5 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [3]:
pd.set_option('display.max_rows', 2000)

In [7]:
picks_dic = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS_DIC.TXT',delimiter='\t')
picks = pd.read_csv('../../SPE_006_originalData/OilSandsDB/PICKS.TXT',delimiter='\t')
wells = pd.read_csv('../../SPE_006_originalData/OilSandsDB/WELLS.TXT',delimiter='\t')
gis = pd.read_csv('../../well_lat_lng.csv')
picks_new=picks[picks['HorID']==13000]
picks_paleoz=picks[picks['HorID']==14000]
df_new = pd.merge(wells, picks_new, on='SitID')
df_paleoz = pd.merge(wells, picks_paleoz, on='SitID')
#### NOTE: This now includes the GIS or well_lat_lng dataset too!
df_gis = pd.merge(df_paleoz, gis, on='SitID')
df_new=pd.merge(df_gis, df_new, on='SitID')
df_new.head()

,SitID,UWI (AGS)_x,UWI_x,HorID_x,Pick_x,Quality_x,UWI (AGS)_y,UWI_y,HorID_y,Pick_y,Quality_y,lat,lng,UWI (AGS),UWI,HorID,Pick,Quality
0,102496,0674010812000,00/12-08-067-01W4/0,14000,561,1,0674010812000,00/12-08-067-01W4/0,13000,475,3,54.785907,-110.129320,0674010812000,00/12-08-067-01W4/0,13000,475,3
1,102497,0674020807000,00/07-08-067-02W4/0,14000,604.5,1,0674020807000,00/07-08-067-02W4/0,13000,515,3,54.782284,-110.269446,0674020807000,00/07-08-067-02W4/0,13000,515,3
2,102498,0674021109000,00/09-11-067-02W4/0,14000,564,1,0674021109000,00/09-11-067-02W4/0,13000,480,3,54.785892,-110.186851,0674021109000,00/09-11-067-02W4/0,13000,480,3
3,102500,0674022910000,00/10-29-067-02W4/0,14000,636.5,1,0674022910000,00/10-29-067-02W4/0,13000,549,3,54.829624,-110.269422,0674022910000,00/10-29-067-02W4/0,13000,549,3
4,102501,0674023406000,00/06-34-067-02W4/0,14000,613,1,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832,0674023406000,00/06-34-067-02W4/0,13000,529,2


## Possible Things to Do with Lat / Long

### Features
1. Find average pick depth of N neighbor wells within a certain maximum distance
2. Return an array of pick depths at N neighbor wells within a certain maximum distance.
3. Return the average depth above or below another known pickin N nearest neighbor wells
3. Return an array of the depths above or below another known pickin N nearest neighbor wells
4. -
5. Return the amount of range in # 2
6. -
7. Return the amount of range in # 4
8. Return if a minimum number of wells are within a certain distance
9. Return the normalized hamming distance between a window around the pick in N nearst wells and window around a point in the well in question
10. Return something about the amount of variance or max/min gr or resititivity within window in question and around pick in neighboring well(s).

###  Mapping
1. Show error on map
2. Show distribution of a feature on a map.
3. Identify where jumps in pick depth occur.
4. Identify where areas of greater variance in depth occur.
5. Identify where different unsupervized groupings occur.


In [8]:
#### 1. Find average pick depth of N neighbor wells within a certain maximum distance
#### takes in array or pandas df? returns dataframe?
def avgPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    return avg_depth
    

In [9]:
#### 2. Return an array of pick depths at N neighbor wells within a certain maximum distance.
#### takes in array or pandas df? returns dataframe?
def arrayOfPickDepthForKneighborsWcertainDist(array,k_neighbors,max_distance):
    return array_of_depths
    

In [10]:
#### 3. Return the average depth above or below another known pickin N nearest neighbor wells
#### takes in array or pandas df? returns dataframe?
def avgDepthAbvOrBelowKneighborsWcertainDist(array,k_neighbors,max_distance,aboveOrbelow,pickA,pickB):
    return avg_of_depthsAbvBelow

In [11]:
#### 4. Return the average depth above or below another known pickin N nearest neighbor wells
#### takes in array or pandas df? returns dataframe?
def arrayDepthAbvOrBelowKneighborsWcertainDist(array,k_neighbors,max_distance,aboveOrbelow,pickA,pickB):
    return array_of_depthsAbvBelow